In [1]:
from bs4 import BeautifulSoup
import re

# Load the file content
with open("aapl.txt", "r", encoding="utf-8") as file:
    html_content = file.read()

# Parse HTML using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Extract table rows
rows = soup.find_all("tr", class_="yf-1jecxey")

# Extract date and closing price from each valid row (ignore dividend/event rows)
data_dict = {}
for row in rows:
    cells = row.find_all("td")
    if len(cells) >= 5 and not cells[1].has_attr("colspan"):
        date = cells[0].get_text(strip=True)
        close_price = cells[4].get_text(strip=True)
        try:
            # Clean and convert price
            close_price = float(close_price.replace(",", ""))
            data_dict[date] = close_price
        except ValueError:
            continue

# Display the first 10 entries
dict(list(data_dict.items())[:10])

{'Apr 4, 2025': 188.38,
 'Apr 1, 2025': 188.38,
 'Mar 1, 2025': 222.13,
 'Feb 1, 2025': 241.84,
 'Jan 1, 2025': 236.0,
 'Dec 1, 2024': 250.42,
 'Nov 1, 2024': 237.33,
 'Oct 1, 2024': 225.91,
 'Sep 1, 2024': 233.0,
 'Aug 1, 2024': 229.0}

In [2]:
from datetime import datetime

# Define what counts as a quarterly date (first of Jan, Apr, Jul, Oct)
quarter_start_months = {"Jan", "Apr", "Jul", "Oct"}

# Filter the dictionary to include only entries where the month is a quarter start
quarterly_data = {
    date: price
    for date, price in data_dict.items()
    if date.split()[0] in quarter_start_months and date.split()[1] == "1,"
}

# Display the first 10 quarterly entries
dict(list(quarterly_data.items())[:10])

{'Apr 1, 2025': 188.38,
 'Jan 1, 2025': 236.0,
 'Oct 1, 2024': 225.91,
 'Jul 1, 2024': 222.08,
 'Apr 1, 2024': 170.33,
 'Jan 1, 2024': 184.4,
 'Oct 1, 2023': 170.77,
 'Jul 1, 2023': 196.45,
 'Apr 1, 2023': 169.68,
 'Jan 1, 2023': 144.29}

In [3]:
# Map months to quarters
month_to_quarter = {
    "Jan": "Q4",
    "Apr": "Q1",
    "Jul": "Q2",
    "Oct": "Q3"
}

# Create a new dictionary with formatted quarter keys
quarterly_data_with_quarters = {}

for date_str, price in quarterly_data.items():
    month_abbr, _, year = date_str.split()
    quarter = month_to_quarter.get(month_abbr)
    if quarter:
        quarter_key = f"{quarter} {year}"
        quarterly_data_with_quarters[quarter_key] = price

# Display first 10 entries
dict(list(quarterly_data_with_quarters.items())[:10])

{'Q1 2025': 188.38,
 'Q4 2025': 236.0,
 'Q3 2024': 225.91,
 'Q2 2024': 222.08,
 'Q1 2024': 170.33,
 'Q4 2024': 184.4,
 'Q3 2023': 170.77,
 'Q2 2023': 196.45,
 'Q1 2023': 169.68,
 'Q4 2023': 144.29}

In [4]:
class QuarterIterator:
    def __init__(self, start_quarter="Q1", start_year=1985):
        self.quarters = ["Q1", "Q2", "Q3", "Q4"]
        self.current_index = self.quarters.index(start_quarter)
        self.year = start_year

    def __iter__(self):
        return self

    def __next__(self):
        quarter = self.quarters[self.current_index]
        result = f"{quarter} {self.year}"
        self.current_index += 1
        if self.current_index >= len(self.quarters):
            self.current_index = 0
            self.year += 1
        return result

['Q1 1985',
 'Q2 1985',
 'Q3 1985',
 'Q4 1985',
 'Q1 1986',
 'Q2 1986',
 'Q3 1986',
 'Q4 1986',
 'Q1 1987',
 'Q2 1987']

In [5]:
portfolio = {
    "CASH": 10000.0,
    "AAPL": 0.0,
    "BONDS": 0.0,
    "INDEX": 0.0,
    "GOLD": 0.0,
    "SAVINGS": 0.0,
}

prices = {
    "AAPL": quarterly_data_with_quarters,
}

current_quarter = QuarterIterator("Q1", 1985)

In [6]:
# Game loop
import sys

round_count = 0

while True:
    command = input("Enter command (next / buy [ASSET] [AMOUNT] / sell [ASSET] [AMOUNT]): ").strip().lower()
    
    if command == "next":
        round_count += 1
        quarter = next(current_quarter)

        print(f"\n📆 Quarter: {quarter}")
        
        # Get prices for this quarter
        current_prices = {k: v.get(quarter, 0.0) for k, v in prices.items()}
        print(f"💹 Current Prices: {current_prices}")
        
        # Update net worth
        net_worth = portfolio["CASH"] + sum(
            portfolio[asset] * current_prices.get(asset, 0)
            for asset in portfolio
            if asset != "CASH"
        )
        print(f"💰 Net Worth: ${net_worth:,.2f}")
        
        # Income every 4 quarters
        if round_count % 4 == 0:
            portfolio["CASH"] += 10000
            print("💸 Received $10,000 income!")

        print(f"📊 Portfolio: {portfolio}")
        
    elif command.startswith("buy "):
        try:
            _, asset, amount = command.split()
            amount = float(amount)
            price = prices.get(asset.upper(), {}).get(quarter, 0)
            
            if price == 0:
                print("⚠️ Price data not available for this asset this quarter.")
                continue

            shares = amount / price
            if portfolio["CASH"] >= amount:
                portfolio["CASH"] -= amount
                portfolio[asset.upper()] += shares
                print(f"✅ Bought {shares:.2f} of {asset.upper()} at ${price:.2f} per unit.")
            else:
                print("❌ Not enough cash.")
        except Exception as e:
            print("❌ Invalid input. Format: buy [ASSET] [AMOUNT]")
    
    elif command.startswith("sell "):
        try:
            _, asset, amount = command.split()
            amount = float(amount)
            asset = asset.upper()
            price = prices.get(asset, {}).get(quarter, 0)

            if price == 0:
                print("⚠️ Price data not available for this asset this quarter.")
                continue

            shares = amount / price
            if portfolio.get(asset, 0) >= shares:
                portfolio[asset] -= shares
                portfolio["CASH"] += amount
                print(f"✅ Sold {shares:.2f} of {asset} at ${price:.2f} per unit.")
            else:
                print("❌ Not enough holdings to sell that much.")
        except Exception as e:
            print("❌ Invalid input. Format: sell [ASSET] [AMOUNT]")

    else:
        print("❓ Unknown command. Try again.")
    
    print("\n" + "-" * 50)

Enter command (next / buy [ASSET] [AMOUNT] / sell [ASSET] [AMOUNT]):  buy aapl 5000


⚠️ Price data not available for this asset this quarter.


Enter command (next / buy [ASSET] [AMOUNT] / sell [ASSET] [AMOUNT]):  next



📆 Quarter: Q1 1985
💹 Current Prices: {'AAPL': 0.09}
💰 Net Worth: $10,000.00
📊 Portfolio: {'CASH': 10000.0, 'AAPL': 0.0, 'BONDS': 0.0, 'INDEX': 0.0, 'GOLD': 0.0, 'SAVINGS': 0.0}

--------------------------------------------------


Enter command (next / buy [ASSET] [AMOUNT] / sell [ASSET] [AMOUNT]):  buy aapl 5000


...

Enter command (next / buy [ASSET] [AMOUNT] / sell [ASSET] [AMOUNT]):  sell aapl 3611


✅ Sold 10317.14 of AAPL at $0.35 per unit.

--------------------------------------------------


Enter command (next / buy [ASSET] [AMOUNT] / sell [ASSET] [AMOUNT]):  next



📆 Quarter: Q2 1990
💹 Current Prices: {'AAPL': 0.38}
💰 Net Worth: $103,611.12
📊 Portfolio: {'CASH': 103611.0, 'AAPL': 0.31746031746661174, 'BONDS': 0.0, 'INDEX': 0.0, 'GOLD': 0.0, 'SAVINGS': 0.0}

--------------------------------------------------


KeyboardInterrupt: Interrupted by user